# Advection Solver

The code solve the Advection equation for different courant factors (array) and grid points (array). It saves everything in files inside the directories, the standard notation is:

method_name/Object_Needed (Solutions, Norms, Tot_var)/filename (N_points and cf)

It will overwrite the files with the same name!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import scipy.stats as sp
import imageio
import os

Clean the directory of the images to provide new snapshots for the video

In [ ]:
# Images for video

# create directory where to save images
print(os.getcwd())

# check if the directory already exist
if not os.path.isdir('./images'):
    os.makedirs('./images')
    
# Empty the 'frames' folder
for file_name in os.listdir('./images'):
    file_path = os.path.join('./images', file_name)
    os.remove(file_path)

os.listdir('./')

# set the directory where your images are stored
directory = "./images/"

# get the list of image files in the directory
files = os.listdir(directory)

# sort the files in alphanumeric order
files = sorted(files)

Parameters for the solution

In [ ]:
# Define the speed of the wave v 
v = 1.0

# Define the domain
L = 10.0    # Domain length
nx = 201    # Number of grid points
method = 'LW'  # Method for the code: FTCS, LF, Leapfrog, LW
initial = 'Gauss' # Initial function: Gauss or Step
boundary = 'Periodic'  # Boundary Conditions: Periodic or Outflow

# Define the final time
t_final = 20.0

# Courant factor: you can pick a vector or a single value
# adding elements will produce all the plots of the code with all cf
cf_vec = [0.1, 0.3, 0.5, 0.8, 1.0]

# Vector of grid points, grid spacing
# Here you can modify the number of points you want to use
# Adding a term will produce the plots of the solution, of the 
# L2 norms and total variations for that number of points
nx_vec = np.array([101, nx, 2*nx, 3*nx, 4*nx, 5*nx])
dx_vec = L / (nx_vec - 1) 
x_vec = [np.linspace(0, L, i) for i in nx_vec]

In [ ]:
# Unfiorm initial function (step function)
def uniform (min_val, max_val, vector):
    
    v1 = np.zeros(len(vector))
    
    for i in range(len(vector)):
        if (vector[i] >= min_val and vector[i] <= max_val):
            v1[i] = 1 
        
    return v1

The algorithm to solve the equation

In [ ]:
def advection_algorithm (method, dx_vec, dt_vec, t_final, x_vec, nx, movie_condition, cf, in_profile, boundary):    
    
    # Inital conditions
    if initial == 'Gauss':   
        x0 = 5
        u_initial_0 = np.exp(-(x_vec[0]-x0)**2)
        u_current = u_initial_0.copy()
        
    elif initial == 'Step':
        u_initial_0 = uniform(4, 6, x_vec[0])
        u_current = u_initial_0.copy()
    
    # Initial conditions for the movie 
    if movie_condition == True:
        plt.plot(x_vec[0], u_current, label = 'initial')
        plt.title('Time='+str(round(0.0,2)))
        plt.ylim(0,1.1)
        plt.savefig('./images/fig_'+ str(0).zfill(5)+'.png', dpi=200)
        plt.close()
    
    
    # Vectors to store the x, y axis of the plots for L2 norms
    L2_plots = []
    time_plots = []
    solution_plots = []
    total_variations_plots = []
    cf_plots_vec = []
    
    for j in range(len(x_vec)):
        
        # initializing the counters
        L2norm_LF = []
        time = []
        total_variations = []
        t = 0.0
        i = 0
        
        # Define the initial condition
        if initial == 'Gauss':
            x0 = 5
            u_initial = np.exp(-(x_vec[j]-x0)**2)
        
            # Initialize the solution array 
            # the 'current' and the 'next' are related to n and n+1 time indeces
            # 'previous' is for n-1
            u_current = u_initial.copy()
            u_previous = u_initial.copy()
        
        elif initial == 'Step':
            u_initial = uniform(4, 6, x_vec[j])
            u_current = u_initial.copy()
            
        # Solving the Advection equation with LF
        if method == 'LF': 
            while t < t_final:
                
            # Compute the new solution using the LF methods 
                u_next = 0.5 * (np.roll(u_current, 1) + np.roll(u_current, -1)) - v*dt_vec[j]/(2*dx_vec[j])*(np.roll(u_current, -1) - np.roll(u_current, 1))    
                
                # Change the buondary condition with the outflow one
                if boundary == 'Outflow': 
                    u_next[0] = 0.5 * (u_current[1] + u_current[0]) - v*dt_vec[j]/(2*dx_vec[j]) * (u_current[1] - u_current[0]) 
                    u_next[-1] = 0.5 * (u_current[-2] + u_current[-1]) - v*dt_vec[j]/(2*dx_vec[j]) * (u_current[-1] - u_current[-2])
                
            # Update the solution
                u_current = u_next.copy()
        
                # advance the time and plot counter
                t += dt_vec[j]
                i += 1
            
                # compute the L2 norm and add the time to the time vector
                L2norm_LF.append(np.sqrt(np.sum(u_current**2)/len(u_current)))
                time.append(t)
                
                # compute the total variations:
                total_variations.append(np.sum(np.abs(u_current - np.roll(u_current, 1))))
                
                # plot the current result and save in an image every 10 iterations
                # You can change the value of j and cf to get the frames of the plot you want.
                if (i % 10 == 0 and j == 1 and cf == 0.5 and movie_condition == True):
                    
                    plt.plot(x_vec[j], u_current)
                    plt.title('Time=' + str(round(t,2)))
                    plt.ylim(0, 1.1)
                    plt.savefig('./images/fig_' + str(i).zfill(5)+ '.png', dpi = 200)
                    plt.close()
               
        # Solving the Advection equation with FTCS
        elif method == 'FTCS':
            while t < t_final:
                
                # Compute the new solution using the FTCS methods
                u_next = u_current - v*dt_vec[j]/(2*dx_vec[j])*(np.roll(u_current, -1) - np.roll(u_current, 1))    
                
                # Update the solution
                u_current = u_next.copy()
        
                # advance the time and plot counter
                t += dt_vec[j]
                i += 1
                
                # compute the L2 norm and add the time to the time vector
                L2norm_LF.append(np.sqrt(np.sum(u_current**2)/len(u_current)))
                time.append(t)
            
                
                if (i % 10 == 0 and j == 0 and cf == 0.5 and movie_condition == True):
                    
                    plt.plot(x_vec[j], u_current)
                    plt.title('Time=' + str(round(t,2)))
                    plt.ylim(0, 1.1)
                    plt.savefig('./images/fig_' + str(i).zfill(5)+ '.png', dpi = 200)
                    plt.close()
        
        # Solving the Advection equation with Leapfrog
        elif method == 'Leapfrog':
            while t < t_final:
                
                if t == 0.0:
                    # Compute the new solution using the LF methods for the first step!
                        u_next = 0.5 * (np.roll(u_current, 1) + np.roll(u_current, -1)) - v*dt_vec[j]/(2*dx_vec[j])*(np.roll(u_current, -1) - np.roll(u_current, 1))    
                        
                        if boundary == 'Outflow': 
                            u_next[0] = 0.5 * (u_current[1] + u_current[0]) - v*dt_vec[j]/(2*dx_vec[j]) * (u_current[1] - u_current[0]) 
                            u_next[-1] = 0.5 * (u_current[-2] + u_current[-1]) - v*dt_vec[j]/(2*dx_vec[j]) * (u_current[-1] - u_current[-2])
                           
                    # Update the solution
                        u_previous = u_current.copy()
                        u_current = u_next.copy()
                        
                else:
                    # Compute the new solution using Leapfrog method
                    u_next = u_previous - v*dt_vec[j]/dx_vec[j] * (np.roll(u_current, -1) - np.roll(u_current, 1))
                    
                        # Not implemented
                    if boundary == 'Outflow': 
                        #u_next[0] = 0.5 * (u_current[1] + u_current[0]) - v*dt_vec[j]/(2*dx_vec[j]) * (u_current[1] - u_current[0]) 
                        u_next[-1] = 0.5 * (u_current[-2] + u_current[-1]) - v*dt_vec[j]/(2*dx_vec[j]) * (u_current[-1] - u_current[-2])
                        
                    # Update the solution
                    u_previous = u_current.copy()
                    u_current = u_next.copy()
                    
                # advance the time and plot counter
                t += dt_vec[j]
                i += 1
                
                # compute the L2 norm and add the time to the time vector
                L2norm_LF.append(np.sqrt(np.sum(u_current**2)/len(u_current)))
                time.append(t)
                
                if (i % 10 == 0 and j == 0 and cf == 0.5 and movie_condition == True):
                    
                    plt.plot(x_vec[j], u_current)
                    plt.title('Time=' + str(round(t,2)))
                    plt.ylim(0, 1.1)
                    plt.savefig('./images/fig_' + str(i).zfill(5)+ '.png', dpi = 200)
                    plt.close()
                    
        # Solving the Advection equation with LW
        elif method == 'LW':
            while t < t_final:
                
                # Compute the new solution using Lax-Wendroff method
                u_next = u_current - (v * dt_vec[j]/(2 * dx_vec[j])) * (np.roll(u_current, -1) - np.roll(u_current, 1)) + (v * dt_vec[j]/dx_vec[j])**2 * 0.5 * (np.roll(u_current, -1) - 2*u_current + np.roll(u_current, 1))
                
                if boundary == 'Outflow': 
                    u_next[0] = u_current[0] - (v * dt_vec[j]/(2 * dx_vec[j])) * (u_current[1] - u_current[0]) + (v * dt_vec[j]/dx_vec[j])**2 * 0.5 * (u_current[1] - 2 * u_current[0] + u_current[0])
                    u_next[-1] = u_current[-1] - (v * dt_vec[j]/(2 * dx_vec[j])) * (u_current[-1] - u_current[-2]) + (v * dt_vec[j]/dx_vec[j])**2 * 0.5 * (u_current[-1] - 2 * u_current[-1] + u_current[-2])
                    
                # Update the solution
                u_current = u_next.copy()
        
                # advance the time and plot counter
                t += dt_vec[j]
                i += 1
                
                # compute the L2 norm and add the time to the time vector
                L2norm_LF.append(np.sqrt(np.sum(u_current**2)/len(u_current)))
                time.append(t)
                
                # compute the total variations:
                total_variations.append(np.sum(np.abs(u_current - np.roll(u_current, 1))))
                
                if (i % 10 == 0 and j == 4 and cf == 0.5 and movie_condition == True):
                    
                    plt.plot(x_vec[j], u_current)
                    plt.title('Time=' + str(round(t,2)))
                    plt.ylim(0, 1.1)
                    plt.savefig('./images/fig_' + str(i).zfill(5)+ '.png', dpi = 200)
                    plt.close()
            
        else:
            print('No method')
            break


        solution_plots.append([x_vec[j], u_current])
    
        # Updating the vectors for the plot of L2 norm
        L2_plots.append(L2norm_LF.copy())
        time_plots.append(time.copy())
        total_variations_plots.append(total_variations.copy())
        print('Max of L2 Norm for each number of points = ', L2_plots[0][0])

        #if os.path.isdir(str(method)):
            
         #   for file_name in os.listdir(str(method) + '/Solutions'):
          #      os.remove(file_name)

           # for file_name in os.listdir(str(method) + '/Norms'):
            #    os.remove(file_name)

            #for file_name in os.listdir(str(method) + '/Tot_var'):
             #   os.remove(file_name)

        os.makedirs(str(method), exist_ok = True)
        os.makedirs(str(method) + '/Solutions', exist_ok = True)
        os.makedirs(str(method) + '/Norms', exist_ok = True)
        os.makedirs(str(method) + '/Tot_var', exist_ok = True)
        
        filename = 'N_points_' + str(len(x_vec[j])) +  '_cf_' + str(cf)
        
        np.savez(os.path.join(str(method) + '/Solutions', filename), 
                x_grid = x_vec[j],
                sol = u_current,
                cf = cf,
                n_points = len(x_vec[j]))
        
        np.savez(os.path.join(str(method) + '/Norms', filename), 
                time = time,
                L2_norm = L2norm_LF,
                cf = cf,
                n_points = len(time))
        
        np.savez(os.path.join(str(method) + '/Tot_var', filename), 
                time = time,
                total_variations = total_variations,
                cf = cf,
                n_points = len(time))
        
    return 


In [ ]:
cf_plots_list = []

# Reperat the method for each value of cf
for i in cf_vec:

    # Vector of timesteps and iteration number vector
    dt_vec = i * dx_vec/v
    n_it_vec = t_final * (dt_vec**-1)
    
    advection_algorithm(method, dx_vec, dt_vec, t_final, x_vec, nx_vec, movie_condition = False, cf = i, in_profile = initial, boundary = boundary)
